In [13]:
import polars as pl
from pathlib import Path

In [5]:
meds = pl.read_parquet('/projectnb/vkolagrp/bellitti/clinicalnotes-databasing-validated/results/medications_tabulated.parquet')

In [10]:
meds.filter(
    (pl.col('standard_ingredient').is_null()) &
    (pl.col('medication').is_not_null())
    )

VAC,medication,standard_ingredient
i64,str,str
2343,"""ZZFISH OIL 1000MG CAP""",null
2181,"""Glucerna""",null
998,"""Valdoxin 50 mg""",null
998,"""Iodoral 5 mg""",null
1461,"""INSULIN SYRINGE 0.5ML 31G 8MM""",null
…,…,…
1067,"""Insulin Syringe 0.5ML""",null
1260,"""Ophthalmic ointment""",null
1557,"""Insulin syringe – 1mL 31g 5/16…",null


In [18]:
german_acb_scale = Path('../data/german_anticholinergic_scale.json')

acb_scale = pl.read_json(german_acb_scale)

In [15]:
import sys
sys.path.append('/projectnb/vkolagrp/bellitti/clinicalnotes-databasing-validated/src')

from standardize_meds import standardize_drug_name

In [25]:
acb_std = acb_scale.with_columns(
    pl.col('drug').map_elements(standardize_drug_name,return_dtype=pl.List(pl.String)).alias('standard_name')
).explode('standard_name')

In [26]:
acb_std

drug,acb_score,standard_name
str,i64,str
"""Aclidinium""",1,"""aclidinium"""
"""Entacapone""",1,"""entacapone"""
"""Mirtazapine""",1,"""mirtazapine"""
"""Alprazolam""",1,"""alprazolam"""
"""Escitalopram""",1,"""escitalopram"""
…,…,…
"""Tizanidine""",3,"""tizanidine"""
"""Tolterodine""",3,"""tolterodine"""
"""Trihexyphenidyl""",3,"""trihexyphenidyl"""
